In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt


In [4]:
import pandas as pd
import random
from faker import Faker

# Khởi tạo Faker để tạo dữ liệu giả
fake = Faker()

# Hàm sinh dữ liệu cho Shopee
def generate_shopee_data(num_records=10):
    data = []
    categories = ['Quần áo', 'Đồ điện tử', 'Máy ảnh', 'Phụ kiện', 'Giày thể thao']
    sentiments = ['Positive', 'Neutral', 'Negative']
    
    for _ in range(num_records):
        record = {
            'ID': fake.uuid4(),
            'Product ID': f"P{random.randint(90000, 99999)}",
            'Customer ID': f"C{random.randint(101, 200)}",
            'Rating': random.randint(1, 5),
            'Comment': fake.sentence(nb_words=10),
            'Date': fake.date_this_year(),
            'Category': random.choice(categories),
            'Sentiment': random.choice(sentiments),
            'Likes': random.randint(0, 100),
            'Dislikes': random.randint(0, 10),
            'Location': fake.city(),
            'Product Features': fake.sentence(nb_words=5)
        }
        data.append(record)
    
    return pd.DataFrame(data)

# Tạo dữ liệu cho Shopee
shopee_data = generate_shopee_data(10)

# Lưu dữ liệu vào file CSV (tùy chọn)
shopee_data.to_csv('shopee_comments.csv', index=False)

# Hiển thị một vài dòng dữ liệu
print("Shopee Data Sample:")
print(shopee_data.head())


Shopee Data Sample:
                                     ID Product ID Customer ID  Rating  \
0  422aa214-ec7b-47fd-9ba9-34a9bc98084d     P95531        C183       3   
1  d91577d2-9315-42f7-a063-1a58624bdf1e     P91287        C175       5   
2  fa15702a-d7db-4feb-b769-84255cc8582d     P99714        C190       4   
3  bcc56f54-c83d-4b18-a24c-41f00c68df58     P99019        C190       2   
4  81b8eb26-d1fb-4d00-b53b-363c3e55df7a     P91560        C132       5   

                                             Comment        Date    Category  \
0  Fight increase sport benefit college create si...  2024-05-27     Quần áo   
1  Indicate up we meet back loss she result hair ...  2024-03-15    Phụ kiện   
2  Baby light unit spend weight act again will se...  2024-04-12  Đồ điện tử   
3                  Laugh draw do school few feeling.  2024-11-16     Máy ảnh   
4  By outside name city none task increase never ...  2024-01-21     Quần áo   

  Sentiment  Likes  Dislikes          Location  \
0  N

In [5]:
import pandas as pd
import random
import numpy as np

# Giả sử bạn đã có DataFrame shopee_data sau khi sinh ra dữ liệu
# Chúng ta sẽ làm tiền xử lý cho dữ liệu này

# 1. Xử lý dữ liệu thiếu
shopee_data = shopee_data.dropna()  # Loại bỏ các dòng có dữ liệu thiếu

# 2. Chuyển đổi cột 'Date' về định dạng datetime
shopee_data['Date'] = pd.to_datetime(shopee_data['Date'], errors='coerce')

# 3. Chuyển cột 'Rating' thành kiểu dữ liệu int (nếu chưa phải)
shopee_data['Rating'] = shopee_data['Rating'].astype(int)

# 4. Loại bỏ các bình luận ngắn (dưới 5 từ chẳng hạn)
shopee_data = shopee_data[shopee_data['Comment'].apply(lambda x: len(x.split()) >= 5)]

# 5. Chuẩn hóa các văn bản trong cột 'Comment' (chuyển thành chữ thường, loại bỏ ký tự đặc biệt)
shopee_data['Comment'] = shopee_data['Comment'].str.lower()  # Chuyển thành chữ thường
shopee_data['Comment'] = shopee_data['Comment'].str.replace(r'\W', ' ', regex=True)  # Loại bỏ ký tự đặc biệt

# 6. Kiểm tra các cột 'Likes' và 'Dislikes' để chắc chắn không có giá trị âm
shopee_data['Likes'] = shopee_data['Likes'].apply(lambda x: max(x, 0))
shopee_data['Dislikes'] = shopee_data['Dislikes'].apply(lambda x: max(x, 0))

# 7. Đảm bảo cột 'Sentiment' là một trong các giá trị hợp lệ
sentiment_values = ['Positive', 'Neutral', 'Negative']
shopee_data = shopee_data[shopee_data['Sentiment'].isin(sentiment_values)]

# 8. Xử lý cột 'Location' để đảm bảo không có giá trị thiếu
shopee_data['Location'] = shopee_data['Location'].fillna('Unknown')

# 9. Đảm bảo các cột số như 'Likes', 'Dislikes' không có giá trị ngoại lệ
shopee_data['Likes'] = shopee_data['Likes'].clip(lower=0, upper=100)  # Giới hạn Likes trong phạm vi hợp lý
shopee_data['Dislikes'] = shopee_data['Dislikes'].clip(lower=0, upper=10)  # Giới hạn Dislikes trong phạm vi hợp lý

# 10. Tạo thêm các cột mới nếu cần, ví dụ: phân tích Sentiment (Nếu bạn muốn phân tích)
# Cột Sentiment đã có sẵn, nếu bạn muốn thực hiện phân tích cảm xúc nâng cao hơn, có thể dùng thư viện như `TextBlob`

# Kiểm tra dữ liệu sau khi tiền xử lý
print(shopee_data.head())


                                     ID Product ID Customer ID  Rating  \
0  422aa214-ec7b-47fd-9ba9-34a9bc98084d     P95531        C183       3   
1  d91577d2-9315-42f7-a063-1a58624bdf1e     P91287        C175       5   
2  fa15702a-d7db-4feb-b769-84255cc8582d     P99714        C190       4   
3  bcc56f54-c83d-4b18-a24c-41f00c68df58     P99019        C190       2   
4  81b8eb26-d1fb-4d00-b53b-363c3e55df7a     P91560        C132       5   

                                             Comment       Date    Category  \
0  fight increase sport benefit college create si... 2024-05-27     Quần áo   
1  indicate up we meet back loss she result hair ... 2024-03-15    Phụ kiện   
2  baby light unit spend weight act again will se... 2024-04-12  Đồ điện tử   
3                  laugh draw do school few feeling  2024-11-16     Máy ảnh   
4  by outside name city none task increase never ... 2024-01-21     Quần áo   

  Sentiment  Likes  Dislikes          Location  \
0  Negative     51         7  

In [9]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Giả sử bạn đã có dữ liệu
texts = ["Tuyệt vời, rất thích!", "Chất lượng không tốt", "Sản phẩm ổn", "Dịch vụ tệ"]
labels = [1, 0, 1, 0]  # 1 = Positive, 0 = Negative

# Tiền xử lý văn bản
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=100)

# Chia dữ liệu thành train/test
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2)

# Đảm bảo dữ liệu X_train và X_test là numpy array
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Xây dựng mô hình CNN
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))  # Sử dụng sigmoid cho phân loại nhị phân

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=5, batch_size=32)

# Đánh giá mô hình
accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy[1]*100:.2f}%')


Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6667 - loss: 0.6960
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.6667 - loss: 0.6555
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.6667 - loss: 0.6231
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.6667 - loss: 0.5966
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.6667 - loss: 0.5731
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step - accuracy: 0.0000e+00 - loss: 0.8988
Accuracy: 0.00%


In [10]:
from keras.layers import LSTM

# Xây dựng mô hình LSTM
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))  # Phân loại nhị phân

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=5, batch_size=32)

# Đánh giá mô hình
accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy[1]*100:.2f}%')


Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.3333 - loss: 0.6982
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.6667 - loss: 0.6809
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.6667 - loss: 0.6681
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.6667 - loss: 0.6605
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.6667 - loss: 0.6416
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 896ms/step - accuracy: 0.0000e+00 - loss: 0.8198
Accuracy: 0.00%


In [11]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Tải tokenizer và mô hình BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Chuẩn bị dữ liệu
texts = ["Tuyệt vời, rất thích!", "Chất lượng không tốt", "Sản phẩm ổn", "Dịch vụ tệ"]
labels = [1, 0, 1, 0]  # 1 = Positive, 0 = Negative

# Tokenize văn bản
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)

# Chia dữ liệu thành train/test
X_train, X_test, y_train, y_test = train_test_split(encodings['input_ids'], labels, test_size=0.2)

# Huấn luyện mô hình
model.compile(optimizer=Adam(learning_rate=2e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(np.array(X_train), np.array(y_train), epochs=3, batch_size=8)

# Đánh giá mô hình
accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f'Accuracy: {accuracy[1]*100:.2f}%')


ModuleNotFoundError: No module named 'transformers'